# Manejo de Datos en PyTorch

## Librerías

In [1]:
import json
import csv
import functools
import gzip
import numpy as np
import pandas as pd
import torch
import tempfile

from gensim.models import KeyedVectors
from gensim.parsing import preprocessing
from torch.utils.data import Dataset, DataLoader, IterableDataset

## La clase Dataset

La clase abstracta [`torch.utils.data.Dataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) es la clase base para construir un dataset de PyTorch. Cualquier dataset personalizado debe heredar de dicha clase e implementar los siguientes métodos:

- `__len__`: Para que `len(dataset)` devuelva el tamaño del conjunto de datos.
- `__getitem__`: Para soportar indexado de manera que `dataset[i]` devuelva el elemento `i`. Es común que en ciertos casos se utilice este método para levantar el dato real (e.g. una imagen) mientras que lo que se guarde en el dataset sea sólo una referencia a dicho dato (e.g. un path a la imagen). De esta manera se evita cargar muchas imágenes en memoria, haciendo que sea menos demandante a nivel RAM.

In [2]:
class IMDBReviewsDataset(Dataset):
    def __init__(self, path, transform=None):
        self.dataset = pd.read_csv(path)
        self.transform = transform
    
    def __len__(self):
        return self.dataset.shape[0]

    def __getitem__(self, item):
        if torch.is_tensor(item):
            item = item.tolist()  # Deal with list of items instead of tensor
        
        item = {
            "data": self.dataset.iloc[item]["review"],
            "target": self.dataset.iloc[item]["sentiment"]
        }

        if self.transform:
            item = self.transform(item)
        
        return item

dataset = IMDBReviewsDataset("./data/imdb_reviews.csv.gz")
print(f"Dataset loaded with {len(dataset)} elements")
print(f"Sample element:\n{dataset[0]}")

Dataset loaded with 50000 elements
Sample element:
{'data': "One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><b

## Transformaciones

El ejemplo anterior nos muestra el uso básico, pero claramente no podemos pasarle eso a una red neuronal, no puede manejar texto. Es para eso que tenemos que hacer algún tipo de transformación sobre los atributos (en este caso el único atributo es el texto). 

### Normalización

En particular, como vemos en el caso anterior, el texto no está normalizado, parte de las transformaciones pueden incluir realizar algún tipo de normalización. Para eso hagamos uso de [`gensim`](https://radimrehurek.com/gensim/index.html), en particular utilizaremos el módulo [`preprocessing`](https://radimrehurek.com/gensim/parsing/preprocessing.html#module-gensim.parsing.preprocessing) que se encargará de hacer varias normalizaciones por defecto

In [3]:
class TextPreprocess:
    def __init__(self, filters=None):
        if filters:
            self.filters = filters
        else:
            self.filters = [
                lambda s: s.lower(),
                preprocessing.strip_tags,
                preprocessing.strip_punctuation,
                preprocessing.strip_multiple_whitespaces,
                preprocessing.strip_numeric,
                preprocessing.remove_stopwords,
                preprocessing.strip_short,
            ]
        
    def _preprocess_string(self, string):
        return preprocessing.preprocess_string(string, filters=self.filters)

    def _encode_target(self, target):
        return 1 if target == "positive" else 0

    def __call__(self, item):
        if isinstance(item["data"], str):
            data = self._preprocess_string(item["data"])
        else:
            data = [self._preprocess_string(d) for d in item["data"]]
        
        if isinstance(item["target"], str):
            target = self._encode_target(item["target"])
        else:
            target = [self._encode_target(t) for t in item["target"]]
        
        return {
            "data": data,
            "target": target
        }

preprocess = TextPreprocess()
print(preprocess(dataset[0]))

{'data': ['reviewers', 'mentioned', 'watching', 'episode', 'hooked', 'right', 'exactly', 'happened', 'thing', 'struck', 'brutality', 'unflinching', 'scenes', 'violence', 'set', 'right', 'word', 'trust', 'faint', 'hearted', 'timid', 'pulls', 'punches', 'regards', 'drugs', 'sex', 'violence', 'hardcore', 'classic', 'use', 'word', 'called', 'nickname', 'given', 'oswald', 'maximum', 'security', 'state', 'penitentary', 'focuses', 'mainly', 'emerald', 'city', 'experimental', 'section', 'prison', 'cells', 'glass', 'fronts', 'face', 'inwards', 'privacy', 'high', 'agenda', 'city', 'home', 'aryans', 'muslims', 'gangstas', 'latinos', 'christians', 'italians', 'irish', 'scuffles', 'death', 'stares', 'dodgy', 'dealings', 'shady', 'agreements', 'far', 'away', 'main', 'appeal', 'fact', 'goes', 'shows', 'wouldn', 'dare', 'forget', 'pretty', 'pictures', 'painted', 'mainstream', 'audiences', 'forget', 'charm', 'forget', 'romance', 'mess', 'episode', 'saw', 'struck', 'nasty', 'surreal', 'couldn', 'ready',

### Conversión a vectores

Podemos continuar convertiendo el texto en una representación por vectores. Si bien hay muchas posibilidades (siendo la bolsa de palabras una de las más utilizadas), en general para Deep Learning se prefieren representaciones utilizando vectores contínuos, obtenidos por algún método del estilo de Word2Vec, Glove o FastText. Para este caso utilizaremos las representaciones de Glove de dimensión 50 que se dejaron para descargar en el [notebook 0](./0_set_up.ipynb).

In [4]:
class VectorizeText:
    def __init__(self, glove_vectors_path):
        self.glove_model = KeyedVectors.load_word2vec_format("./data/glove.6B.50d.txt",
                                                             binary=False, no_header=True)
        self.unkown_vector = np.random.randn(self.glove_model.vector_size)  # Random vector for unknown words
    
    def _get_vector(self, word):
        if word in self.glove_model:
            return self.glove_model[word]
        else:
            return self.unkown_vector
    
    def _get_vectors(self, sentence):
        return np.vstack([self._get_vector(word) for word in sentence])
    
    def __call__(self, item):
        review = []
        if isinstance(item["data"][0], str):
            review = self._get_vectors(item["data"])
        else:
            review = [self._get_vectors(d) for d in item["data"]]

        return {
            "data": review,
            "target": item["target"]
        }

vectorizer = VectorizeText("./data/glove.6B.50d.txt.gz")
vectorizer(preprocess(dataset[0]))

{'data': array([[-0.18105   , -0.79229999, -0.097616  , ...,  1.42859995,
         -0.032471  ,  0.47235999],
        [ 0.69395   ,  0.69261003, -0.21608   , ...,  0.2247    ,
         -0.23197   ,  0.0062523 ],
        [-0.0049087 ,  0.12611   ,  0.14056   , ..., -0.58464003,
         -0.31830999,  0.31564   ],
        ...,
        [ 0.25435999, -0.44304001, -0.12524   , ...,  0.73352998,
          0.026198  ,  0.30408001],
        [-0.058468  ,  0.019087  ,  0.089056  , ..., -0.28176001,
          0.045137  , -0.18802001],
        [ 0.14443   ,  0.39103001, -0.93454999, ..., -0.71325999,
         -0.54575998,  0.13952   ]]),
 'target': 1}

### Combinación de vectores

Si bien ahora estamos con una versión de los atributos que podría pasar por una red neuronal, hay un problema, las distintas reviews tienen largo distinto y como el algoritmo se entrena en lotes (*mini-batches*) estas requieren tener todas el mismo largo. Hay varias maneras de lidiar con esto, cada una con sus ventajas y desventajas. Dado que por ahora solo vimos perceptrón multicapa, que espera algo de tamaño fijo, una opción sencilla puede ser la de simplemente promediar los vectores de palabras.

In [5]:
class WordVectorsAverage:
    def __call__(self, item):
        if item["data"][0].ndim == 2:
            data = np.vstack([np.mean(d, axis=0) for d in item["data"]])
        else:
            data = np.mean(item["data"], axis=0)
        
        return {
            "data": data,
            "target": item["target"]
        }

vector_average = WordVectorsAverage()
vector_average(vectorizer(preprocess(dataset[0])))

{'data': array([ 0.09577639,  0.02841517, -0.10557132, -0.27231854,  0.26206218,
         0.06613897, -0.06309304, -0.0415889 , -0.15025635,  0.20153827,
        -0.20744573, -0.10961742, -0.15226319, -0.01454608,  0.27548899,
         0.01196117,  0.03742069, -0.00949933, -0.12544634, -0.2413244 ,
        -0.05297129,  0.30832897,  0.15259472,  0.18935842,  0.11253999,
        -1.07986028, -0.36013918,  0.19516013,  0.42554837, -0.16026932,
         2.04100643,  0.14682758, -0.05537557, -0.37286566, -0.0821818 ,
         0.1196377 , -0.05145981, -0.15015274, -0.19053322, -0.23536209,
        -0.13351049,  0.06572485,  0.04443282,  0.26679263,  0.12758326,
         0.0022425 ,  0.00446468, -0.1353851 , -0.0229578 , -0.06999757]),
 'target': 1}

### Conversión de vectores a tensores

En el paso final, debemos convertir nuestros datos de arrays de `numpy` a tensores de PyTorch.

In [6]:
class ToTensor:
    def __call__(self, item):
        """
        This espects a single array.
        """
        return {
            "data": torch.from_numpy(item["data"]),
            "target": torch.tensor(item["target"])
        }

to_tensor = ToTensor()
to_tensor(vector_average(vectorizer(preprocess(dataset[0]))))

{'data': tensor([ 0.0958,  0.0284, -0.1056, -0.2723,  0.2621,  0.0661, -0.0631, -0.0416,
         -0.1503,  0.2015, -0.2074, -0.1096, -0.1523, -0.0145,  0.2755,  0.0120,
          0.0374, -0.0095, -0.1254, -0.2413, -0.0530,  0.3083,  0.1526,  0.1894,
          0.1125, -1.0799, -0.3601,  0.1952,  0.4255, -0.1603,  2.0410,  0.1468,
         -0.0554, -0.3729, -0.0822,  0.1196, -0.0515, -0.1502, -0.1905, -0.2354,
         -0.1335,  0.0657,  0.0444,  0.2668,  0.1276,  0.0022,  0.0045, -0.1354,
         -0.0230, -0.0700], dtype=torch.float64),
 'target': tensor(1)}

### Componiendo las transformaciones

Para evitar tener que llamar a todas las funciones de transformación que querramos aplicar, para ello hacemos uso del parámetro `transform` que definimos en nuestro `Dataset` y un poco de ayuda de `functools`.

In [7]:
def compose(*functions):
    return functools.reduce(lambda f, g: lambda x: g(f(x)), functions, lambda x: x)

dataset = IMDBReviewsDataset("./data/imdb_reviews.csv.gz",
                             transform=compose(preprocess, vectorizer, vector_average, to_tensor))
print(f"Dataset loaded with {len(dataset)} elements")
print(f"Sample element:\n{dataset[0]}")

Dataset loaded with 50000 elements
Sample element:
{'data': tensor([ 0.0958,  0.0284, -0.1056, -0.2723,  0.2621,  0.0661, -0.0631, -0.0416,
        -0.1503,  0.2015, -0.2074, -0.1096, -0.1523, -0.0145,  0.2755,  0.0120,
         0.0374, -0.0095, -0.1254, -0.2413, -0.0530,  0.3083,  0.1526,  0.1894,
         0.1125, -1.0799, -0.3601,  0.1952,  0.4255, -0.1603,  2.0410,  0.1468,
        -0.0554, -0.3729, -0.0822,  0.1196, -0.0515, -0.1502, -0.1905, -0.2354,
        -0.1335,  0.0657,  0.0444,  0.2668,  0.1276,  0.0022,  0.0045, -0.1354,
        -0.0230, -0.0700], dtype=torch.float64), 'target': tensor(1)}


### Iterando el dataset

Ya tenemos nuestro conjunto de datos con sus respectivas transformaciones. ¿Para qué nos sirve esto? Una opción es simplemente iterar en el conjunto de datos de a un elemento. Esto es sencillo, simplemente se hace a través de un `for`.

In [8]:
for idx, sample in enumerate(dataset):
    print(sample["data"])
    print(sample["target"])
    print("=" * 50)
    
    if idx == 2:
        break

tensor([ 0.0958,  0.0284, -0.1056, -0.2723,  0.2621,  0.0661, -0.0631, -0.0416,
        -0.1503,  0.2015, -0.2074, -0.1096, -0.1523, -0.0145,  0.2755,  0.0120,
         0.0374, -0.0095, -0.1254, -0.2413, -0.0530,  0.3083,  0.1526,  0.1894,
         0.1125, -1.0799, -0.3601,  0.1952,  0.4255, -0.1603,  2.0410,  0.1468,
        -0.0554, -0.3729, -0.0822,  0.1196, -0.0515, -0.1502, -0.1905, -0.2354,
        -0.1335,  0.0657,  0.0444,  0.2668,  0.1276,  0.0022,  0.0045, -0.1354,
        -0.0230, -0.0700], dtype=torch.float64)
tensor(1)
tensor([ 0.0651,  0.1982, -0.3791, -0.1193,  0.3404,  0.1934, -0.2716, -0.1303,
        -0.1434,  0.3831, -0.0742,  0.2368, -0.1542,  0.1296,  0.2765, -0.0633,
         0.1217,  0.1036, -0.1228, -0.3544,  0.0960,  0.3598,  0.0418,  0.0399,
         0.3906, -0.7744, -0.5847,  0.1390,  0.3948, -0.1475,  2.1128, -0.1157,
         0.0859, -0.3700,  0.0146,  0.3372, -0.0167,  0.2811, -0.2000, -0.3213,
         0.1288,  0.1174, -0.1521,  0.0546, -0.0459,  0.0700, 

## La clase Dataloader

El problema con iterar de a un elemento es que estamos limitados al querer entrenar un modelo. Por empezar, los modelos de Deep Learning suelen ser más eficientes si se entrenan utilizando algún tipo de entrenamiento por *mini-batches*. Además, hay otras cosas como mezclar los elementos (*shuffling*) o cargar datos en paralelo vía distintos *multiprocess workers*. La clase [`torch.utils.data.DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) precisamente se encarga de hacer eso por nosotros:

In [9]:
dataloader = DataLoader(
    dataset,
    batch_size=4,
    shuffle=True,
    num_workers=2
)

for i_batch, sample_batched in enumerate(dataloader):
    print(i_batch, 
          sample_batched['data'].size(),
          sample_batched['target'].size())

    if i_batch == 2:
        print(sample_batched["data"])
        print(sample_batched["target"])
        break

0 torch.Size([4, 50]) torch.Size([4])
1 torch.Size([4, 50]) torch.Size([4])
2 torch.Size([4, 50]) torch.Size([4])
tensor([[ 8.4240e-02,  2.4439e-01, -1.7102e-01, -2.3076e-02,  8.9962e-02,
          1.4342e-01, -1.6285e-01, -1.8584e-01, -1.4967e-01,  3.5144e-01,
         -2.0516e-01,  6.0792e-02, -1.6507e-01,  1.6065e-01,  2.5354e-01,
         -9.5613e-02,  1.2308e-01,  6.9195e-03, -3.6781e-01, -3.9112e-01,
          2.2053e-01,  6.2114e-02,  1.2022e-01, -1.6964e-02,  2.0150e-01,
         -7.3657e-01, -5.6199e-01,  1.3429e-01,  1.6235e-01, -1.4231e-01,
          1.9862e+00,  6.4501e-02,  1.4963e-01, -2.3589e-01,  1.6971e-01,
          6.5741e-02,  7.0174e-02,  1.5929e-01, -2.4950e-01, -2.6196e-01,
         -1.3692e-01,  1.6855e-01, -1.1381e-01, -1.0460e-01,  2.7676e-03,
         -5.4849e-02,  1.7385e-01, -1.8558e-01,  3.3055e-02,  2.2113e-01],
        [ 1.3920e-01,  4.0550e-02, -1.7777e-01, -6.4388e-02,  1.2557e-01,
          1.2271e-01, -1.6567e-01, -1.6975e-01, -9.9027e-02,  2.2501e-0

## La clase IterableDataset

El método preferido para trabajar con conjuntos de datos en PyTorch es `torch.utils.data.Dataset`. En general, hacer uso inteligente del método `__getitem__`, e.g. usándolo para cargar imágenes a medida que sean necesitadas y no al instanciar el dataset, es la mejor manera de trabajar con un conjunto de datos. En particular, de esta forma es mucho más fácil hacer *shuffling* de los datos y demás. No obstante, no siempre esto es posible, muchas veces el conjunto de datos es demasiado grande para levantarlo en memoria (aunque sólo levantemos referencias). Para esos casos, PyTorch ofrece la clase [`torch.utils.data.IterableDataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.IterableDataset), en este caso el único método que es requerido implementar es `__iter__`.

In [5]:
class MeLiChallengeDataset(IterableDataset):
    def __init__(self, path, transform=None):
        self.dataset_path = path
        self.transform = transform

    def __iter__(self):
        with gzip.open(self.dataset_path, "rt") as fh:
            for l in fh:
                data = json.loads(l)
                item = {
                    "data": data['title'],
                    "target": data['category']
                }
                
                if self.transform:
                    yield self.transform(item)
                else:
                    yield item

dataset = MeLiChallengeDataset("./practico/data/meli-challenge-2019/spanish.train.jsonl.gz")
dataloader = DataLoader(dataset, batch_size=4, shuffle=False, num_workers=0)
dataiter = iter(dataloader)
print(f"Sample batch:\n{dataiter.next()}")

<class '__main__.MeLiChallengeDataset'>
Sample batch:
{'data': ['Casita Muñecas Barbies Pintadas', 'Neceser Cromado Holográfico ', 'Funda Asiento A Medida D20 Chevrolet', 'Embrague Ford Focus One 1.8 8v Td (90cv) Desde 01-99'], 'target': ['DOLLHOUSES', 'TOILETRY_BAGS', 'CAR_SEAT_COVERS', 'AUTOMOTIVE_CLUTCH_KITS']}


In [6]:
print('Propiedades del objeto dataset:')
keys = dataset.__dict__.keys()
for key in keys:
    print('-', key)

Propiedades del objeto dataset:
- dataset_path
- transform
